<a href="https://colab.research.google.com/github/Avniiii2606/Weather-Prediction/blob/main/weather_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'weather-chatbot-dataset-intent-entity:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F487151%2F908036%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240801%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240801T102816Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8c020bf6440036e28f558ead71a087c45578032eb0d8025d5bbaf52cad1d76280c2f469233c6c36d260f2a2938ef854e46ae83d13bb69798dc59ff96d8ddfae2b3adcc634d71ced79e15fec29a18b606dadc8e6f249caf02d4ef11b2f002129795dc5331c4ead1382c36346b4331ccfbf44c8560379a8714d90c2804d4ae759c820176cb04db272e08eeb2631303c779afc6bf70b1efac09a184c389c7c182050970c697178dff21f25c2b98676ff17cd4200e950435fff7f922ed7e9de534ebdd312d618a46011305d5097c74d6d5223325fe734e8810c8bfefbb8b8bf38a587fcb6e1158aacd31edfa5296fe1db2282cdd6c561ec7901126a1d9dd6b9792e0'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 91421 bytes downloaded
Downloaded and uncompressed: weather-chatbot-dataset-intent-entity
Data source import complete.


In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/weather-chatbot-dataset-intent-entity/weather_intent_entities.json


In [ ]:
import pandas as pd

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import GlobalMaxPooling1D, Dense, Activation, Dropout, Embedding,Conv1D
import random

In [ ]:
def load_json_file(filename):
    with open(filename) as f:
        file = json.load(f)
    return file

filename = '/kaggle/input/weather-chatbot-dataset-intent-entity/weather_intent_entities.json'

intents = load_json_file(filename)

In [ ]:
intents

{'rasa_nlu_data': {'common_examples': [{'entities': [{'start': 25,
      'entity': 'timeRange',
      'end': 34,
      'value': 'this year'},
     {'start': 38,
      'entity': 'geographic_poi',
      'end': 81,
      'value': 'Horseshoe Lake State Fish and Wildlife Area'}],
    'intent': 'GetWeather',
    'text': 'What will the weather be this year in Horseshoe Lake State Fish and Wildlife Area?'},
   {'entities': [{'start': 11,
      'entity': 'condition_description',
      'end': 16,
      'value': 'sunny'},
     {'start': 17,
      'entity': 'timeRange',
      'end': 54,
      'value': 'one hundred thirty five days from now'},
     {'start': 58,
      'entity': 'geographic_poi',
      'end': 96,
      'value': 'Monterey Bay National Marine Sanctuary'}],
    'intent': 'GetWeather',
    'text': 'Will it be sunny one hundred thirty five days from now in Monterey Bay National Marine Sanctuary'},
   {'entities': [{'start': 18,
      'entity': 'condition_description',
      'end': 22,
  

In [ ]:
df = pd.DataFrame(columns=['text','intent'])

In [ ]:
df

,text,intent


In [ ]:
def extract_json_info(json_file, df):
    # Iterate over each intent in the JSON file
    for intent in json_file['rasa_nlu_data']['common_examples']: # Access intents under 'rasa_nlu_data'
        # Iterate over each pattern in the current intent
        for pattern in intent['text']: # Assuming 'text' holds the patterns
            # Create a list containing the pattern and its associated tag
            sentence_tag = [pattern, intent.get('intent', 'text')] # Handle cases where 'intent' might be missing
            # Append the pattern and tag to the DataFrame
            df.loc[len(df.index)] = sentence_tag
     # Return the updated DataFrame
    return df

df = extract_json_info(intents, df)
df.head(20)

,text,intent
0,W,GetWeather
1,h,GetWeather
2,a,GetWeather
3,t,GetWeather
4,,GetWeather
5,w,GetWeather
6,i,GetWeather
7,l,GetWeather
8,l,GetWeather
9,,GetWeather


In [ ]:
labels = df['entity'].unique().tolist()
labels = [s.strip() for s in labels]

In [ ]:
labels

['W',
 'h',
 'a',
 't',
 '',
 'w',
 'i',
 'l',
 'e',
 'r',
 'b',
 's',
 'y',
 'n',
 'H',
 'o',
 'L',
 'k',
 'S',
 'F',
 'd',
 'f',
 'A',
 '?',
 'u',
 'v',
 'm',
 'M',
 'B',
 'N',
 'c',
 'I',
 'p',
 '0',
 "'",
 'g',
 '1',
 ',',
 '2',
 '3',
 '9',
 'K',
 'P',
 'J',
 'T',
 'C',
 '.',
 '4',
 'G',
 'z',
 'Å',
 'R',
 'U',
 'O',
 ':',
 '7',
 '6',
 '5',
 '8',
 'é',
 '-',
 'j',
 'V',
 'D',
 'x',
 'E',
 '/',
 '',
 'q',
 'Z',
 'ō',
 'ã',
 'í',
 'Y',
 'ê',
 '–',
 'Q',
 'X',
 'ú',
 'à',
 'ū',
 '’']

In [ ]:
df.shape

(97733, 2)

In [ ]:
len(df)

97733

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97733 entries, 0 to 97732
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   entity  97733 non-null  object
 1   value   97733 non-null  object
dtypes: object(2)
memory usage: 2.2+ MB


In [ ]:
counts = df['entity'].value_counts()
print(counts)

entity
     16637
e     9713
t     7696
a     6554
i     6214
     ...  
ã        1
ú        1
à        1
ū        1
’        1
Name: count, Length: 82, dtype: int64


In [ ]:
len(labels)

82

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['value'])

# Convert text patterns to numerical sequences
X = tokenizer.texts_to_sequences(df['value'])

# Pad sequences to ensure uniform length
max_sequence_length = max(len(seq) for seq in X)
X_padded = pad_sequences(X, maxlen=max_sequence_length, padding='post')

# Convert entity to numerical labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['entity'])

In [ ]:
X_padded

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

In [ ]:
y

array([41, 52, 45, ...,  7, 10, 13])

In [ ]:
print(X_padded.shape)
print(y.shape)

(97733, 1)
(97733,)


In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_padded, y, test_size=0.2, random_state=42)

In [ ]:
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words
    pattern_words = doc[0]
    # convert pattern_words in lower case
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create bag of words array,if word match found in current pattern then put 1 otherwise 0.[row * colm(263)]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # in output array 0 value for each tag ang 1 value for matched tag.[row * colm(8)]
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle training and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test. X - patterns(words), Y - intents(tags)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


NameError: name 'classes' is not defined

In [ ]:
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200, batch_size=32)

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(set(y)))
y_val = tf.keras.utils.to_categorical(y_val, num_classes=len(set(y)))

In [ ]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words
    pattern_words = doc[0]
    # convert pattern_words in lower case
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create bag of words array,if word match found in current pattern then put 1 otherwise 0.[row * colm(263)]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # in output array 0 value for each tag ang 1 value for matched tag.[row * colm(8)]
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle training and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test. X - patterns(words), Y - intents(tags)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

NameError: name 'Sequential' is not defined

In [ ]:
{'start': 25,
      'entity': 'timeRange',
      'end': 34,
      'value': 'this year'},

